In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
from tensorflow.keras import backend as K
import tensorflow_datasets as tfds
from tensorflow.train import BytesList, FloatList, Int64List
from tensorflow.train import Feature, Features, Example
import numpy as np
import time

# 9.

In [ ]:
train_set, valid_set, test_set = tfds.load(
    'mnist',
    split=['train[:50000]', 'train[50000:]', 'test'],
    as_supervised=True,
    shuffle_files=True,
)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:
class RBFLayer(layers.Layer):
    def __init__(self, units, gamma, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.units = units
        self.gamma = K.cast_to_floatx(gamma)

    def build(self, input_shape):
        self.mu = self.add_weight(name='mu',
                                  shape=(int(input_shape[1]), self.units),
                                  initializer='uniform',
                                  trainable=True)
        super(RBFLayer, self).build(input_shape)

    def call(self, inputs):
        diff = K.expand_dims(inputs) - self.mu
        l2 = K.sum(K.pow(diff,2), axis=1)
        res = K.exp(-1 * self.gamma * l2)
        return res

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.units)

In [ ]:
def normalize(image):
    return tf.cast(image, tf.float32) / 255.


model = keras.models.Sequential([
    layers.Input((28, 28, 1)),
    layers.Lambda(normalize),
    layers.ZeroPadding2D(padding=2),
    layers.Conv2D(filters=6, kernel_size=5, activation='tanh'),
    layers.AveragePooling2D(pool_size=2, strides=[2, 2]),
    layers.Conv2D(filters=16, kernel_size=5, activation='tanh'),
    layers.AveragePooling2D(pool_size=2, strides=[2, 2]),
    layers.Conv2D(filters=120, kernel_size=5, activation='tanh'),
    layers.Flatten(),
    layers.Dense(84, activation='tanh'),
    RBFLayer(10, 0.5)
])
model.compile(
    optimizer='nadam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
batch_size = 128
train_set = train_set.batch(batch_size).prefetch(1)
test_set = test_set.batch(batch_size).prefetch(1)
valid_set = valid_set.batch(batch_size).prefetch(1)

In [ ]:
model.fit(
    train_set,
    epochs=10,
    validation_data=valid_set
)
model.evaluate(test_set)

Epoch 1/10
391/391 [==============================] - 9s 15ms/step - loss: 0.4904 - accuracy: 0.8854 - val_loss: 0.1457 - val_accuracy: 0.9587
Epoch 2/10
391/391 [==============================] - 3s 7ms/step - loss: 0.1089 - accuracy: 0.9694 - val_loss: 0.0850 - val_accuracy: 0.9762
Epoch 3/10
391/391 [==============================] - 3s 7ms/step - loss: 0.0695 - accuracy: 0.9803 - val_loss: 0.0718 - val_accuracy: 0.9787
Epoch 4/10
391/391 [==============================] - 3s 7ms/step - loss: 0.0527 - accuracy: 0.9854 - val_loss: 0.0651 - val_accuracy: 0.9805
Epoch 5/10
391/391 [==============================] - 3s 6ms/step - loss: 0.0424 - accuracy: 0.9881 - val_loss: 0.0602 - val_accuracy: 0.9810
Epoch 6/10
391/391 [==============================] - 3s 7ms/step - loss: 0.0346 - accuracy: 0.9907 - val_loss: 0.0568 - val_accuracy: 0.9823
Epoch 7/10
391/391 [==============================] - 3s 7ms/step - loss: 0.0287 - accuracy: 0.9925 - val_loss: 0.0544 - val_accuracy: 0.9838
Epoch

[0.04711465537548065, 0.9868000149726868]

# 10.

In [ ]:
@tf.function
def preprocess(image, label):
    resized_image = tf.image.resize(image, (224, 224))
    final_image = keras.applications.vgg19.preprocess_input(resized_image)
    return final_image, label


def augment(dataset, length):
    augmentations = [
        lambda image: tf.image.flip_left_right(image),
        lambda image: tf.image.flip_up_down(image),
        lambda image: tf.image.stateless_random_contrast(
            image, 0.2, 0.5, seed=(42, 42)
        ),
        lambda image: tf.image.stateless_random_saturation(
            image, 0.5, 1.0, seed=(42, 42)
        ),
        lambda image: tf.image.stateless_random_brightness(
            image, 0.2, seed=(42, 42)
        ),
        lambda image: tf.image.stateless_random_crop(
            image, size=(1, 2, 3), seed=(42, 42)
        )
    ]
    augmented_sets = []
    for augmentation in augmentations:
        augmented_sets.append(dataset.map(
            lambda image, label: (augmentation(image), label)
        ))

    updated_dataset = dataset
    for augmented_set in augmented_sets:
        updated_dataset = updated_dataset.concatenate(augmented_set)

    new_length = length*(len(augmented_sets)+1)
    return updated_dataset, new_length


In [ ]:
K.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

(train_set, valid_set, test_set), ds_info = tfds.load(
    'oxford_flowers102',
    split=['train', 'validation', 'test'],
    as_supervised=True,
    shuffle_files=True,
    with_info=True
)
train_len = ds_info.splits['train'].num_examples
n_classes = ds_info.features['label'].num_classes
batch_size = 8

train_set, train_len = augment(train_set, train_len)
train_set = train_set.map(preprocess).shuffle(train_len).batch(batch_size).prefetch(1)
test_set = test_set.map(preprocess).batch(batch_size).prefetch(1)
valid_set = valid_set.map(preprocess).batch(batch_size).prefetch(1)

In [ ]:
base_model = keras.applications.VGG19(weights='imagenet', include_top=False)

avg = layers.GlobalAveragePooling2D()(base_model.output)
output = layers.Dense(n_classes, activation='softmax')(avg)
#model = keras.Model(inputs=base_model.input, outputs=output)

n_original_layers = len(base_model.layers)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
drive_path = 'gdrive/MyDrive/MachineLearning/HandsOnMachineLearning/chapter14'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
model = keras.models.load_model(f'{drive_path}/models/checkpoint_2024_06_17-15_39_04')

In [ ]:
import gc


class GarbageCollectorCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        K.clear_session()
        gc.collect()

In [ ]:
# RAM usage increasing every epoch can be caused by running in eager mode

In [ ]:
for layer in model.layers[:n_original_layers]:
    layer.trainable = False

filepath = time.strftime(f"{drive_path}/models/freezed_original_layers_%Y_%m_%d-%H_%M_%S")
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    filepath,
    monitor='val_loss',
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch',
    initial_value_threshold=None
)
optimizer = keras.optimizers.SGD(learning_rate=0.2, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
model.fit(train_set, epochs=1, validation_data=valid_set,
          callbacks=[GarbageCollectorCallback(), checkpoint_cb])

In [ ]:
for layer in model.layers[:n_original_layers]:
    layer.trainable = True

filepath = time.strftime(f"{drive_path}/models/modified_original_layers%Y_%m_%d-%H_%M_%S")
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    filepath,
    monitor='val_loss',
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch',
    initial_value_threshold=None
)
# possible improvement using differential learning rates
optimizer = keras.optimizers.SGD(learning_rate=0.00001, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
model.fit(train_set, epochs=1, validation_data=valid_set,
          callbacks=[GarbageCollectorCallback(), checkpoint_cb])

# Dataset from a TFRecord file

In [ ]:
def img_dataset_to_file(filename, dataset):
    with tf.io.TFRecordWriter(f'{filename}.tfrecord') as f:
        for image, label in dataset:
            raw_image = tf.io.serialize_tensor(image)
            img_example = Example(
                features=Features(
                    feature={
                        'image': Feature(bytes_list=BytesList(value=[raw_image.numpy()])),
                        'label': Feature(int64_list=Int64List(value=[label]))
                    }
                )
            )
            f.write(img_example.SerializeToString())


img_dataset_to_file('train-set', train_set)
feature_description = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.int64),
}


def _parse_function(proto):
    example = tf.io.parse_single_example(proto, feature_description)
    image = tf.io.parse_tensor(example['image'], out_type=tf.float32)
    label = example['label']
    return image, label


def image_generator(tfrecord_file):
    raw_dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = raw_dataset.map(_parse_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    for batch in dataset.batch(batch_size):
        yield batch


output_signature = (
    tf.TensorSpec(shape=(None, None, None, 3), dtype=tf.float32),
    tf.TensorSpec(shape=(None,), dtype=tf.int64)  # Labels
)
train_set = tf.data.Dataset.from_generator(
    lambda: image_generator('train-set.tfrecord'),
    output_signature=output_signature
)